<a href="https://colab.research.google.com/github/afvallejo/Workshop_New_frontiers/blob/main/1_QC_to_Annotation_workshop_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions
**Quick start**

1. Setup your analysis conditions using the sliders under setup.
2. Press "Runtime" -> "Run all".
3. The pipeline will run automatically up to an annotated umap plot

**If you get an error**

*   Press "Runtime" -> "Factory reset  runtime" and then try againg the quick start instructions.



# Installing packages

In [ ]:
#@title Install
!pip install scanpy[louvain] anndata2ri leidenalg scrublet

# Setup

In [ ]:
#@title Setup

state = 42
metric = "euclidean"


# Filtering criteria
min_counts = 500#@param {type:"slider", min:50, max:1000, step:50}
max_counts = 40000#@param {type:"slider", min:5000, max:50000, step:5000}
min_genes = 5#@param {type:"slider", min:5, max:100, step:5}

mito_criteria = 20#@param {type:"slider", min:0, max:100, step:5}

flavor="cell_ranger" #@param ["cell_ranger","seurat"] {allow-input: true}
n_top_genes = 3000#@param {type:"slider", min:500, max:5000, step:100}

n_neighbors = 30#@param {type:"slider", min:5, max:50, step:5}
num_PCA = 30#@param {type:"slider", min:5, max:50, step:5}


#_________________________________________________________________________________



# Load packages

In [ ]:
#@title load 
import scanpy as sc
import scanpy.external as sce

# numpy et al.
import numpy as np
import scipy.sparse as sp
import pandas as pd
import gc

# R integration
from rpy2.robjects.packages import importr

from rpy2.robjects.vectors import StrVector, FloatVector, ListVector
import rpy2.robjects as ro
import anndata2ri

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import seaborn as sb
import re
import scipy as sp
import datetime, time

pl.rcParams['pdf.fonttype'] = 'truetype'
from matplotlib import colors
sc.set_figure_params(vector_friendly=False,dpi_save=300,transparent=True)
pl.rcParams['lines.linewidth'] = 0.1
sc.set_figure_params(color_map='Reds')


sc.settings.verbosity = 3               # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()
sc.logging.print_version_and_date()



# Setup

## setup  Working Directory WD

In [ ]:
# print your current directory 
!pwd

In [ ]:
!mkdir PBMC
%cd PBMC

In [ ]:
# change here to the folder containig the Aligned data
folder='/content/PBMC'

# change here for the name of your sample, this name will be added to the figures names.
samplename="sample1"

#set the WD as in "folder" and create a directory for figures
import os
os.chdir(folder)
if not os.path.exists('./figures'):
    os.makedirs('./figures')

# Download test data from 10X

In [ ]:
# Download the data from the 10x website
!wget https://cf.10xgenomics.com/samples/cell-exp/3.1.0/connect_5k_pbmc_NGSC3_ch1/connect_5k_pbmc_NGSC3_ch1_filtered_feature_bc_matrix.tar.gz


In [ ]:
# unpack the downloaded files
!tar xvfz connect_5k_pbmc_NGSC3_ch1_filtered_feature_bc_matrix.tar.gz

In [ ]:
# print current WD, check is correct!
!pwd

# Load the data

## MTX from cell ranger

After loading the data, metadata can be added. In this example sample names, treatment and donor are used as metadata. This labels can be used for gouping samples or perform statistical tests.
 

In [ ]:
adata = sc.read_10x_mtx("/content/PBMC/filtered_feature_bc_matrix")
adata.obs['sample'] = ['sample1']*adata.n_obs
adata.obs['treatment'] = ['CTR']*adata.n_obs
adata.obs['donor'] = ['D1']*adata.n_obs
#adata.X = sp.sparse.csr_matrix(adata.X)
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata

 # QC and filtering

Data quality control can be split into cell QC and gene QC. Typical quality measures for assessing the quality of a cell include the number of molecule counts (UMIs), the number of expressed genes, and the fraction of counts that are mitochondrial. A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration.


# Filter doublets with scrublet

Scrublet simulates doublets from the observed data and uses a k-nearest-neighbor classifier to calculate a continuous doublet_score (between 0 and 1) for each transcriptome. The score is automatically thresholded to generate predicted_doublets, a boolean array that is True for predicted doublets and False otherwise.

**Best practices:**

- When working with data from multiple samples, run Scrublet on each sample separately. Because Scrublet is designed to detect technical doublets formed by the random co-encapsulation of two cells, it may perform poorly on merged datasets where the cell type proportions are not representative of any single sample.

- Check that the doublet score threshold is reasonable (in an ideal case, separating the two peaks of a bimodal simulated doublet score histogram, as in this example), and adjust manually if necessary.

- Visualize the doublet predictions in a 2-D embedding (e.g., UMAP or t-SNE). Predicted doublets should mostly co-localize (possibly in multiple clusters). If they do not, you may need to adjust the doublet score threshold, or change the pre-processing parameters to better resolve the cell states present in your data.

# Filter doublets with scrublet

In [ ]:
sce.pp.scrublet(adata,
    adata_sim = None,
    sim_doublet_ratio= 2.0,
    expected_doublet_rate = 0.05,
    stdev_doublet_rate = 0.02,
    synthetic_doublet_umi_subsampling= 1.0,
    knn_dist_metric = 'euclidean',
    normalize_variance= True,
    log_transform= False,
    mean_center= True,
    n_prin_comps= 30,
    use_approx_neighbors= True,
    get_doublet_neighbor_parents= False,
    n_neighbors = None,
    threshold = None,
    verbose = True,
    copy= False,
random_state= 0,)

In [ ]:
sce.pl.scrublet_score_distribution(adata)

## remove doublets

In [ ]:
adata = adata[adata.obs.predicted_doublet == False]
adata

# QC filtering

This code will calculate the QC covariates:

- total number of counts per cell
- number of expressed genes per cell
- fraction of mitochondrial reads per cell

Note: mitochondrial genes in human start with 'MT-'

In general it is a good idea to be permissive in the early filtering steps, and then come back to filter out more stringently when a clear picture is available of what would be filtered out. 


In [ ]:
# Quality control - calculate QC covariates
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)

#mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
adata.obs['mt_frac'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)

#adata.obs['mt_frac'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
adata

#mt_gene_mask = [gene.startswith('MT-') for gene in adata.var['gene_symbol']]
#adata.obs['mt_frac'] = adata.X[:, mt_gene_mask].sum(1)/adata.obs['n_counts']

## Filtering

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = min_counts)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = max_counts)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < mito_criteria]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = min_genes)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

In [ ]:
sb.set_context('paper')
pl.rcParams['lines.linewidth'] = 0.1
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4), gridspec_kw={'wspace':0.2})

ax1_dict = sc.pl.violin(adata, 'n_counts', groupby='sample', size=0.5, log=True, cut=0, ax=ax1, show=False)
ax2_dict =  sc.pl.violin(adata, 'n_genes', groupby='sample', size=0.5, log=True, cut=0, ax=ax2, show=False)
ax3_dict = sc.pl.violin(adata, 'mt_frac', groupby='sample',size=0.5, ax=ax3, show=False,)
savefig='figures/'+samplename + '_1_QC_violin_plots.pdf'
fig.savefig(savefig, dpi=300, bbox_inches='tight')

In [ ]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

In [ ]:
# plot percentage of mitochondtial genes versus count depth and n_genes
# compute qc metrics
regex = re.compile('^(MT).*', re.IGNORECASE)
mito_genes = [l for l in adata.var_names for m in [regex.search(l)] if m]
adata.var['mito'] = False
adata.var.loc[mito_genes, 'mito'] = True
print('Found {} mito genes and annotated.'.format(len(mito_genes)))

sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], inplace=True)

pl.rcParams['figure.figsize']=(5,5) #rescale figures
sc.pl.scatter(adata, x='total_counts', y='n_genes', color='mt_frac',save='.pdf')

# Normalization

In [ ]:

sc.pp.normalize_total(adata, target_sum=1e4,exclude_highly_expressed=True,inplace=True)

## Removal of Mitochondrial and Ribosomal Protein Genes

In [ ]:
ribo_genes = adata.var_names.str.startswith(("RPS","RPL"))
mito_genes = adata.var_names.str.startswith('MT-')
malat1 = adata.var_names.str.startswith('MALAT1')
remove = np.add(ribo_genes,mito_genes)
remove = np.add(remove,malat1)
kept_genes = np.invert(remove)

In [ ]:
adata = adata[:,kept_genes]

In [ ]:
adata.raw = adata
sc.pp.log1p(adata)

# Sellection of HVG

In [ ]:
#Expects logarithmized data.
sc.pp.highly_variable_genes(adata, flavor=flavor, n_top_genes=n_top_genes)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
pl.rcParams['lines.linewidth'] = 0.1
sc.pl.highly_variable_genes(adata,save='.pdf')

## PCA

In [ ]:

sc.pp.scale(adata,max_value=10)
sc.pp.pca(adata, n_comps = 60, use_highly_variable = True, svd_solver = "arpack")
sc.pl.pca_variance_ratio(adata, n_pcs = 40)

# Clustering

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
sc.pp.neighbors(adata, n_pcs=num_PCA,n_neighbors=n_neighbors,random_state=state)
sc.tl.leiden(adata,random_state=42, resolution = 0.8)
sc.tl.umap(adata,random_state=42)
sc.tl.tsne(adata,random_state=42)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata)


In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
genes_to_plot = ['n_genes','log_counts','mt_frac']
savefig="QC_mito.pdf"
sc.pl.tsne(adata, color = genes_to_plot,ncols=2,cmap='viridis',save=savefig)

In [ ]:

sb.set_context('paper')
pl.rcParams['figure.figsize']=(7,7)
#sc.tl.leiden(adata,random_state=random_state, resolution = 0.5)
savefig=samplename+"_clustering_hires_vst.pdf"
sc.pl.umap(adata, color=['leiden'], legend_loc='on data',legend_fontoutline=3,legend_fontsize='small', legend_fontweight='normal',frameon=False,save=savefig)

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(15,10)
fig_ind=np.arange(231, 237)
fig = pl.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6)

p10 = sc.pl.pca_scatter(adata, color='leiden', ax=fig.add_subplot(fig_ind[0]), show=False)
p11 = sc.pl.tsne(adata, color='leiden', ax=fig.add_subplot(fig_ind[1]), show=False)
p12 = sc.pl.umap(adata, color='leiden', ax=fig.add_subplot(fig_ind[2]), show=False)

pl.show()

In [ ]:
pl.rcParams['figure.figsize']=(5,5)
sb.set_context('paper')
sc.tl.leiden(adata, resolution=0.5, key_added='leiden_r0.5')

#Visualize the clustering and how this is reflected by different technical covariates
sc.pl.umap(adata, color=['leiden_r0.5','leiden', ])
#sc.pl.umap(adata, color=['log_counts', 'mt_frac'])

In [ ]:
adata.write('Clustered_data.h5ad')

# Marker genes

In [ ]:
#method : {‘logreg’, ‘t-test’, ‘wilcoxon’, ‘t-test_overestim_var’} | None (default: None)
sc.tl.rank_genes_groups(adata, 'leiden', method='logreg',n_genes=adata.shape[1])
markers=pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20)
markers

In [ ]:
savetable=samplename+"_marker_genes_leiden_10.csv"
markers.to_csv(savetable)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test', groups= ['7'], reference='11',key_added = "wilcoxon")
#sc.tl.rank_genes_groups(adata, 'leiden_r1', method='t-test',key_added = "wilcoxon")
sc.get.rank_genes_groups_df(adata, group='7', key='wilcoxon',log2fc_min=2)

In [ ]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY','NCAM1', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

In [ ]:
sb.set_context('paper')
pl.rcParams['figure.figsize']=(4,4)

sc.pl.umap(adata=adata, color=marker_genes, use_raw=False)

# Visualization

In [ ]:

ax = sc.pl.dotplot(adata, marker_genes, groupby='leiden', use_raw=False)

In [ ]:
sc.pl.heatmap(adata=adata, var_names=marker_genes,
              groupby='leiden', use_raw=False, vmin=0,cmap='viridis')

In [ ]:
ax = sc.pl.stacked_violin(adata, marker_genes, groupby='leiden', use_raw=False)

In [ ]:
adata.obs['annotated'] = adata.obs['leiden'].cat.add_categories(['CD4 T cells', 
                        'Monocytes', 'B cells', 'CD8 T cells', 
                        'FCGR3A+ Monocytes', 'NK cells', 'Dendritic cells'])

adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['1','4'])] = 'CD4 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['5','9'])] = 'B cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['2','3','8','6'])] = 'CD8 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['0','10','11','12','13'])] = 'Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['7',])] = 'NK cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['11', '12'])] = 'Dendritic cells'

adata.obs['annotated'] = adata.obs['annotated'].cat.remove_unused_categories()

In [ ]:

sb.set_context('talk')
pl.rcParams['figure.figsize']=(7,7)
#sc.tl.leiden(adata,random_state=random_state, resolution = 0.5)
savefig=samplename+"_clustering_hires_vst.pdf"
sc.pl.umap(adata, color=['annotated'], legend_loc='on data',legend_fontoutline=3,legend_fontsize='small', legend_fontweight='normal',frameon=False,save=savefig)


In [ ]:
adata.write('Annotated_data.h5ad')